In [52]:
from konlpy.tag import Kkma
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
import numpy as np
 



In [53]:
class SentenceTokenizer(object):
    def __init__(self):
        self.kkma = Kkma()
        self.twitter = Okt()
        self.stopwords = ['중인' ,'만큼', '마찬가지', '꼬집었', "연합뉴스", "데일리", "동아일보", "중앙일보", "조선일보", "기자"
        ,"아", "휴", "아이구", "아이쿠", "아이고", "어", "나", "우리", "저희", "따라", "의해", "을", "를", "에", "의", "가",]
    def text2sentences(self, text):
        sentences = self.kkma.sentences(text)
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences
    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence is not '':
                nouns.append(' '.join([noun for noun in self.twitter.nouns(str(sentence))
                                       if noun not in self.stopwords and len(noun) > 1]))
        return nouns

<>:17: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:17: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-53-5b4175f38dde>:17: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if sentence is not '':


In [54]:
class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer()
        self.graph_sentence = []
        
    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        return  self.graph_sentence
 

In [55]:
class Rank(object):
    def get_ranks(self, graph, d=0.85): # d = damping factor
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 # diagonal 부분을 0으로 
            link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
            if link_sum != 0:
                A[:, id] /= link_sum
            A[:, id] *= -d
            A[id, id] = 1
            
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
        return {idx: r[0] for idx, r in enumerate(ranks)}


In [56]:
class TextRank(object):
    def __init__(self, text):
        self.sent_tokenize = SentenceTokenizer()
        
        self.sentences = self.sent_tokenize.text2sentences(text)
        
        self.nouns = self.sent_tokenize.get_nouns(self.sentences)
                
        self.graph_matrix = GraphMatrix()
        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)        
        
        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)
    
        
    def summarize(self, sent_num=3):
        summary = []
        index=[]
        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)
        
        index.sort()
        for idx in index:
            summary.append(self.sentences[idx])
        
        return summary
        

In [59]:
news1 = """ 국내 신종 코로나바이러스 감염증(코로나19) 신규 확진자가 하루만에 600명대로 올라섰다.전국적으로 크고 작은 일상 감염이 잇따르는 상황에서 해외유입 변이 바이러스도 빠르게 확산되고 있어 당국이 촉각을 세우고 있다. 정부는 오는 21일 다음주부터 적용할 사회적 거리두기 조정안을 발표할 예정이다.19일 중앙방역대책본부(중대본)에 따르면 이날 0시 기준 코로나19 신규 확진자는 654명으로 전날(528명)보다 126명 늘었다.누적 확진자는 13만3471명이다.확진자 수는 보통 주말·휴일 검사건수 감소 영향으로 인해 주 초반에는 비교적 적게 나오다가 중반부터 증가하는 경향을 보인다.이날 신규 확진자의 감염경로는 지역발생이 637명, 해외유입이 17명이다.지역별로는 서울 245명, 경기 159명, 인천 23명 등 수도권이 427명(67%)이다.주요 집단발병 사례를 보면 노래연습장, 유흥업소 등 다중이용시설을 고리로 새로운 감염이 확인됐다.수도권에서는 서울 강동구 노래연습장과 관련해 41명, 서울 노원구 고시원에서 11명, 경기 성남시 일가족-지인 사례에서 19명이 각각 양성 판정을 받았다.앞서 발생한 인천국제공항 검역소 집단감염과 관련해선 8명이 인도 변이에 감염된 것으로 확인됐다. 역학적 관련성이 있는 나머지 7명까지 포함하면 15명 전원이 인도 변이 감염자인 셈이다.최근 코로나19 발생 양상을 보면 전국에서 집단감염이 잇따르면서 '4차 유행'이 지속하고 있다.지난 13일부터 이날까지 최근 1주일간 하루 평균 신규 확진자는 약 651명꼴이다. 이중 '사회적 거리두기' 단계 조정의 핵심 지표인 일평균 지역발생 확진자는 약 628명으로, 여전히 2.5단계(전국 400∼500명 이상 등) 범위다.사망자는 전날보다 8명 늘어 누적 1912명이 됐다. 국내 평균 치명률은 1.43%다.전날 하루 선별진료소를 통한 검사 건수는 3만3640건으로, 직전일(4만1704건)보다 19% 적다.검사건수 대비 확진자를 계산한 양성률은 1.94%(3만3640명 중 654명)로, 직전일 1.27%(4만1704명 중 528명)보다 상승했다.정부는 오는 21일 다음주부터 3주간 적용할 사회적 거리두기 조정안을 발표한다. 현재 수도권에서는 2단계, 비수도권에서는 1.5단계의 거리두기 조처가 이뤄지고 있다."""
news2 = """경북 영천에서 17일 하루 만에 신종 코로나바이러스 감염증(코로나19) 확진자가 5명이 발생, N차 감염이 우려되는 가운데 비상이 걸렸다. 지난 7일 80번 확진자 이후 열흘 만에 지역 누적 확진자는 85명으로 늘어났다.영천시에 따르면 경산에 주소를 둔 공부방 교사가 확진됐다는 소식을 접한 A여고 한 학생이 지난 16일 보건소 검체 검사 결과 17일 양성판정을 받았다.또 확진자 교사가 운영 중인 공부방에서 과외 학습을 받고 있는 B여고 학생이 17일 오후 8시 확진 판정을 받은 가운데 A여고 학생 부모와 B여고 학생 엄마 등 총 5명이 코로나 판정을 받았다.수업을 받은 나머지 학생들은 모두 음성 판정을 받은 것으로 알려졌다.이날 A여고 측은 학생, 교직원 등 140여 명이 보건소에서 검체를 실시했으며 뒤늦게 알려진 B여고는 18일 학생과 교직원들이 검사를 받을 예정이다.이에 보건소는 코로나 확진자들을 자가 격리하고 이들의 이동 동선 파악에 집중하는 가운데 학교 등의 방역소독을 철저히 하고 있다."""
news3 =""" 브라질, 인도발 코로나 변이 상륙할라…긴장 속 모니터링 강화 (상파울루=연합뉴스) 김재순 특파원 = 브라질 보건당국이 인도발 신종 코로나바이러스 감염증(코로나19) 변이 바이러스 상륙 가능성에 긴장하고 있다.18일(현지시간) 브라질 언론에 따르면 상파울루시 당국은 인도발 변이 바이러스가 브라질에서도 나타날 가능성이 크다는 지적에 따라 상파울루주 정부 산하 부탄탕연구소와 함께 모니터링을 강화하기로 했다고 밝혔다.상파울루시는 브라질에서 코로나19 확진자와 사망자가 가장 먼저 나온 곳으로, 인근 과룰류스 국제공항을 통해 외국인 입국이 대규모로 이뤄지고 있다.상파울루시 관계자는 이날 기자회견을 통해 "인도발 변이 바이러스 상륙을 막기 위한 사전 조치로 부탄탕연구소와 협력해 외국인에 대한 철저한 추적 관찰이 이뤄질 것"이라고 말했다."""
news3 = """한강에서 사망한 채 발견된 손정민씨의 실종 당일 함께 술을 마셨던 A씨 측이 대규모 고소를 예고했다. 여기에 사건에 대한 진실 규명을 요구하는 단체가 경찰과 A씨의 휴대전화를 발견한 환경미화원을 검찰에 고발하면서 이른바 ‘한강 대학생 사건’이 고소·고발전으로 번지고 있다. 해당 단체는 추가 고발까지 예고했다.A씨 측 반격 "수만명 고소해야" 손씨의 사망 원인을 밝히기 위한 경찰 수사가 마무리 단계에 접어들었지만 고소·고발에 집회까지 이어지면서 혼란이 가중되고 있다. 5일 A씨 측 등에 따르면 정병원 변호사(법무법인 원앤파트너스)는 A씨와 그 가족 등에 대한 허위사실을 제기한 유튜버와 블로거 등을 7일부터 경찰에 고소할 예정이다.A씨를 대리하는 정 변호사의 고소 대상에는 유튜버와 블로거뿐 아니라 인터넷 커뮤니티 등에 게시글이나 댓글을 작성한 이들도 포함될 예정이다. 정 변호사는 “수차례 친구 A 및 그 가족과 주변인들에 관한 위법행위를 멈춰달라고 요청했음에도 게시물이 오히려 늘어나고 있다”며 “일부 내용은 수인한도를 넘어서면서 피해와 고통은 점점 더 심해지고 있다”고 밝혔다. 이어 “선처를 희망하는 사람이 전혀 없다면 최소 수만 명은 고소해야 할 것 같다”고 덧붙였다.이들은 지난달 29일에도 집회를 열고 손씨의 실종 당일 목격자의 제보를 독려하는 집회를 열어다. 지난 1일에는 서울 종로구 서울경찰청 앞에서 기자회견을 열고 “수사당국이 경찰 초동수사 부실 논란과 손씨 사망 경위에 대한 의혹을 피해왔다”고 주장했다. 반진사를 만든 건 유튜브 ‘종이의 TV’ 채널 운영자다. A씨 측이 명예훼손 등 혐의로 고소하겠다고 밝힌 유튜버 중엔 '종이의 TV‘도 포함돼 있다.손정민 친구 "수만명 고소"에 "끝까지 간다"…혼란의 한강 사망"""
sents = """ 인도에 이어 베트남에서 신종 코로나 바이러스 감염증(코로나19) 변이 바이러스가 확산함에 따라 현지에서 대규모 생산시설을 운영하고 있는 삼성전자와 LG전자에도 비상이 걸렸다. 베트남 지방정부가 코로나19 확산 방지를 위해 이동제한 등의 방역조치를 시행하고 있어 각 시설 생산 차질 우려가 나오는 것이다.3일(현지시각) 베트남 보건당국에 따르면 북부 박장성(北江省)과 박닌성(北寧省) 지역에서는 전날 각각 48명, 3명의 신규 확진자가 발생했다. 지난 4월 27일 이후 전날까지 이 지역 누적 확진자는 박장성 2424명, 박닌성 879명으로, 같은 기간 베트남 전체 확진자(4549명)의 72.6%를 차지하고 있다.이곳에는 삼성전자를 비롯한 전자 계열사들이 대규모 생산시설을 운영 중에 있다. 박닌성 옌퐁에는 약 2만명이 일하고 있는 삼성전자의 스마트폰 공장이 있는데, 이 공장은 삼성전자의 전 세계 스마트폰 출하량의 절반 이상을 담당하고 있는 것으로 알려졌다. 또 삼성디스플레이(직원수 약 3만5000명)와 삼성SDI(2400명)도 해당 지역에서 각각 디스플레이 패널, 배터리 공장을 운영하고 있다.삼성전자는 지난달 박장성 협력업체 직원 40여명이 코로나19에 집단 감염돼 공장 가동이 멈추는 바람에 부품 공급에 애를 먹었다. 이 상황에서 스마트폰 공장의 현지 근로자 2명이 지난달 12일 확진 판정을 받기도 했다. 다만 이들은 유행이 본격화한 4월 말부터 휴가 중으로, 공장 출근을 하지 않아 다른 직원과의 접촉은 없었던 것으로 전해졌다. 공장 가동 중단도 이뤄지지 않았다.여기에 베트남 각 지역 정부들이 1일 오전 0시(현지시각)을 기해 해당 지역에 이동제한 명령을 내리면서 생산 차질이 일부 우려되는 상황이다. 72시간 이내의 코로나19 진단검사에서 음성이 확인될 경우에는 출퇴근이 가능하다고 하지만, 업무에는 상당한 영향이 아닐 수 없다. 현재 삼성전자 스마트폰 공장 직원 2만명 중 절반쯤이 외부 지역에서 출퇴근을 하고 있는 것으로 알려졌다. 삼성전자 관계자는 “현재 공장 내 기숙사에 들어가면 2주간 외부 출입을 하지 못하도록 조치하고 있다”라며 “최대한 (직원들이) 이동하지 않게끔 인근 숙박시설을 마련하는 등 대책을 마련하고 있다”고 했다.공장 내 집단감염이 현실화 할 경우 생산차질로 인한 피해는 막심할 수밖에 없다. 특히 보통 글로벌 기업의 대규모 공장 인근에는 이들의 부품을 조달하는 협력사들도 많아 코로나19 확산은 자칫 공급망 전체의 위기로 번질 가능성이 있다. 국내 중소기업의 베트남 현지 공장 150여곳이 최근 코로나19 확산 여파로 가동을 멈추기도 했다.이 때문에 삼성전자는 현지 직원을 대상으로 한 백신 접종에도 돌입했다. 전자 계열사인 삼성디스플레이와 삼성SDI 등의 직원들도 함께 백신을 맞는다. 해당 백신은 베트남 정부가 기업별로 할당한 영국 아스트라제네카 백신으로, 삼성 계열 3사의 접종 대상자는 1만5000여명이라는 게 회사 측 설명이다. 이들의 협력사에도 백신이 지원될 것으로 알려졌다. 회사 관계자는 “우선 생산라인의 현지인 직원들을 대상으로 백신 접종을 시작했다”라며 “한국인 주재원들의 접종 신청도 받고 있다”고 했다.문제는 최근 베트남에서 영국 변이 바이러스와 인도 변이 바이러스의 특성이 섞인 신종 변이 바이러스가 발견됐다는 점이다. 영국 변이는 전염성이 높아 확산이 빠르고, 인도 변이는 코로나19를 무력화하는 중화항체가 잘 듣지 않는다. 응우옌 탄 롱 베트남 보건부 장관은 “새 변이 바이러스는 공기를 통해 빠르게 전파되며, 배양 실험에서 이전 바이러스보다 더 빠르게 자기 복제가 이뤄지는 모습이 관찰됐다”고 밝혔다.코로나19가 확산하는 지역은 아니지만 역시 베트남에 생산거점을 두고 있는 LG전자도 현지 상황을 면밀하게 관찰하고 있다. LG전자는 베트남 북부 하이퐁시에 지난 2015년 ‘LG 하이퐁 캠퍼스’를 설립하고, 다양한 생활가전 제품을 생산하고 있다. 현지 직원은 약 1만6000명이다. 자동차 부품을 만드는 공장도 있다. 또 LG디스플레이, LG이노텍 등의 계열사 공장이 있다.LG전자 관계자는 “공장의 일부 생산라인의 가동률을 조정하고, 박장성이나 박닌성으로부터 출퇴근하는 근로자의 임시 거처 숙소비를 지원하는 등의 대응을 짜고 있다”라며 “최악의 상황에 대비한 대응 전략을 갖추고 있다”고 했다. LG디스플레이와 LG이노텍 역시 비슷한 대책을 세우고 있다. LG디스플레이와 이노텍 측은 “현지는 아직 코로나19의 급격한 확산이 일어나지는 않고 있으나, 상황을 예의주시하고 있다”고 했다. 코로나19 급속 확산 베트남…삼성·LG, 공급망 무너질까 노심초사."""
textrank = TextRank(news3)
for row in textrank.summarize(3):
    print(row)
    print()




씨 측 반격 " 수 만명 고소해야" 손 씨의 사망 원인을 밝히기 위한 경찰 수사가 마무리 단계에 접어들었지만 고소· 고발에 집회까지 이어지면서 혼란이 가중되고 있다.

씨를 대리하는 정 변호사의 고소 대상에는 유튜버와 블 로 거뿐 아니라 인터넷 커뮤니티 등에 게시 글이나 댓 글을 작성한 이들도 포함될 예정이다.

손 정민 친구 " 수 만명 고소 "에 " 끝까지 간다"… 혼란의 한강 사망

